# Towards Learning Convolutions from Scratch

### Yassin Alnuaimee, Mehrad Sahebi and Danilo Cammarata

The following code snipets are probably a good start for the project

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Load and preprocess the dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# Define the neural network structure (simplified example)
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 1000) # Adjust sizes as per the paper's architecture
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)  # Flatten the image
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = SimpleNet()

# Define the β-lasso training algorithm (simplified pseudo-code)
def beta_lasso_train(net, trainloader):
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(10):  # Loop over the dataset multiple times
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Implement the β-lasso algorithm specifics here
            # ...

# Train the network
beta_lasso_train(net, trainloader)


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data


In [2]:
import torch.optim as optim

def beta_lasso_loss(model, beta):
    """
    Compute the β-lasso regularization term.
    This function computes the sum of the absolute values of the weights,
    scaled by the β parameter.
    """
    reg_loss = 0.0
    for param in model.parameters():
        reg_loss += torch.sum(torch.abs(param))
    return beta * reg_loss

def adjust_beta(initial_beta, epoch, adjustment_factor):
    """
    Dynamically adjust the β parameter.
    """
    return initial_beta * (adjustment_factor ** epoch)

def train_model(model, train_loader, initial_beta, adjustment_factor, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(epochs):
        beta = adjust_beta(initial_beta, epoch, adjustment_factor)
        for inputs, labels in train_loader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Add the β-lasso regularization term
            loss += beta_lasso_loss(model, beta)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

In [ ]:
# Example usage
model = SimpleNet()
train_loader = # Load your data here
initial_beta = 0.01  # Initial β value
adjustment_factor = 0.95  # Factor to adjust β every epoch
train_model(model, train_loader, initial_beta, adjustment_factor)